In [1]:
# Imports
import copy
import networkx as nx
import numpy as np
import pandas
import seaborn; seaborn.set()

In [88]:
class Model(object):
    def __init__(self,graph,initial_infected=[[366],[1000]],reproduction_number=.65,effective_contacts=15,prob_of_flying_day=.0005,passengers_per_flight=150):
        #set pararms
        self.g=graph
        self.initial_infected=initial_infected
        self.reproduction_number=reproduction_number
        self.prob_of_flying_day=prob_of_flying_day
        self.prob_of_flying=1-(1-self.prob_of_flying_day)**30
        self.effective_contacts=effective_contacts
        self.transmission_rate=self.reproduction_number/self.effective_contacts
        self.passengers_per_flight=passengers_per_flight
        
        #set state variables
        self.t=0
        self.history_infected=[]
        self.history_cities_infected=[]
        self.history_infected_interactions=[]
        self.history_space=[]
        
        #self call setup
        self.setup_space()
        
    def setup_space(self):
        self.num_infected=0
        self.num_hosts_infected=0
        for node_id in self.g.nodes():
            if node_id in self.initial_infected[0]:
                self.g.node[node_id]['infected']=self.initial_infected[1][0]
                self.g.node[node_id]["state"] = "I"
                self.num_hosts_infected+=1
                self.num_infected+=self.g.node[node_id]['infected']
            else:
                self.g.node[node_id]['infected']=0
                self.g.node[node_id]["state"] = "S"
        
        # Track the latest step
        self.history_space.append(copy.deepcopy(self.g))
        self.history_infected.append(self.num_infected)
        self.history_cities_infected.append(self.num_hosts_infected)
        self.history_infected_interactions=[]
        
    def step(self):
        self.g_step=copy.deepcopy(self.g)
        # Iterate over I and infect any S neighbors
        for self.node_id in self.g.nodes():
            if self.g.node[self.node_id]["state"] == "I":
                #print self.node_id
                # Decide how many infected are traveling
                self.infected_traveling=np.random.binomial(self.g.node[self.node_id]['infected'],self.prob_of_flying)

                #check
                if self.infected_traveling>self.g.node[self.node_id]['infected']:
                    self.infected_traveling=self.g.node[self.node_id]['infected']
                    
                # Get travel options
                self.neighbors = self.g.neighbors(self.node_id)
                #print('neighbors',neighbors)
                self.passenger_list=[]
                for self.neighbor_id in self.neighbors:
                    self.passenger_list.append(self.g.edge[self.node_id][self.neighbor_id]['weight'])

                #print('passenger_list',passenger_list)
                # Create passenger distribution
                self.percent_list = [float(i)/sum(self.passenger_list) for i in self.passenger_list]

                #passenger_dist=np.cumsum(percent_list)
                #print('passenger_dist',percent_list)

                # Create travel distribution
                self.travel_dist=np.random.multinomial(self.infected_traveling, self.percent_list)
                #print('travel_dist',travel_dist)

                # Convert number of travels to infection
                self.infected_travelers=[]
                for i in xrange(len(self.travel_dist)):
                    self.moving_passengers=self.travel_dist[i]
                    self.max_passengers=self.g.edge[self.node_id][self.neighbors[i]]['weight']
                    self.moving_passengers*=self.transmission_rate*self.passengers_per_flight
                    self.moving_passengers=int(self.moving_passengers)
                    if self.moving_passengers>self.max_passengers:
                        self.moving_passengers=self.max_passengers
                        
                    self.infected_travelers.append(self.moving_passengers)    
                #print self.infected_travelers
                
                # Node update
                # Calculate in host spread
                print self.node_id,'initial',self.g.node[self.node_id]['infected']
                self.new_infected=int(self.g.node[self.node_id]['infected']*self.reproduction_number)
                print self.node_id,'spread',self.new_infected
                # Calculate in host death
                self.removed_infected=self.g.node[self.node_id]['infected']
                print self.node_id,'removed',self.removed_infected
                print self.node_id,'flying',sum(self.infected_travelers)
                # Spread to neighbors
                for i in xrange(len(self.neighbors)):
                    self.neighbor_id=self.neighbors[i]

                    # Move Infected
                    self.num_infected_at_neighbor=self.g.node[self.neighbor_id]['infected']
                    self.g_step.node[self.neighbor_id]['infected']=self.num_infected_at_neighbor+self.infected_travelers[i]
                    self.g_step.node[self.node_id]['infected']=self.g.node[self.node_id]['infected']-self.infected_travelers[i]

                # Sum new infection change
                self.new_infected_total=int(max(self.g.node[self.node_id]['infected']+self.new_infected-self.removed_infected,0))
                self.g_step.node[self.node_id]['infected']=int(self.new_infected_total)
                print self.node_id,'new infected total',self.new_infected_total
                print self.node_id,'new infected total graph',self.g_step.node[self.node_id]['infected']
                
        # Update graph
        self.num_infected=0
        self.num_hosts_infected=0
        for self.node_id in self.g_step.nodes():
            if self.g_step.node[self.node_id]['infected']>0:
                print self.node_id,'pre-count number infected',self.g_step.node[self.node_id]['infected']
                self.g_step.node[self.node_id]["state"] = "I"
                self.num_hosts_infected+=1
                self.num_infected+=self.g_step.node[self.node_id]['infected']
                print self.node_id,'number infected',self.g_step.node[self.node_id]['infected']
            else:
                self.g_step.node[self.node_id]["state"] = "S"

        # Resolve g_step to g ***Not Resolving***
        self.g=self.g_step

        # Track the latest step
        self.history_space.append(copy.deepcopy(self.g))
        self.t+=1
        self.history_infected.append(self.num_infected)
        self.history_cities_infected.append(self.num_hosts_infected)
        self.history_infected_interactions=[]
        


In [89]:
#TEST
# Initialize Space
g=nx.read_weighted_edgelist("2010citytocity_monthly.csv",nodetype=str,delimiter=',')
g=nx.convert_node_labels_to_integers(g,label_attribute='old_label')
g=g.to_directed()

m=Model(g,reproduction_number=.4)
m.step()
print m.history_infected
m.step()
print m.history_infected


366 initial 1000
366 spread 400
366 removed 1000
366 flying 52
366 new infected total 400
366 new infected total graph 400
212 pre-count number infected 12
212 number infected 12
366 pre-count number infected 400
366 number infected 400
396 pre-count number infected 40
396 number infected 40
[1000, 452]
212 initial 12
212 spread 4
212 removed 12
212 flying 0
212 new infected total 4
212 new infected total graph 4
366 initial 400
366 spread 160
366 removed 400
366 flying 24
366 new infected total 160
366 new infected total graph 160
396 initial 40
396 spread 16
396 removed 40
396 flying 0
396 new infected total 16
396 new infected total graph 16
212 pre-count number infected 16
212 number infected 16
366 pre-count number infected 400
366 number infected 400
396 pre-count number infected 16
396 number infected 16
[1000, 452, 432]


In [90]:
m.step()
print m.history_infected

212 initial 16
212 spread 6
212 removed 16
212 flying 0
212 new infected total 6
212 new infected total graph 6
366 initial 400
366 spread 160
366 removed 400
366 flying 32
366 new infected total 160
366 new infected total graph 160
396 initial 16
396 spread 6
396 removed 16
396 flying 4
396 new infected total 6
396 new infected total graph 6
212 pre-count number infected 28
212 number infected 28
255 pre-count number infected 4
255 number infected 4
366 pre-count number infected 400
366 number infected 400
396 pre-count number infected 6
396 number infected 6
[1000, 452, 432, 438]
